In [1]:
pip install transformers datasets datasets[audio] accelerate peft tensorboard evaluate rouge_score bert_score -q

  DEPRECATION: Building 'rouge_score' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'rouge_score'. Discussion can be found at https://github.com/pypa/pip/issues/6334
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -U torchcodec torch torchaudio torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 57.7 MB/s  0:00:12m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.3/322.3 MB 97.1 MB/s  0:00:03m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.7/124.7 MB 99.3 MB/s  0:00:01m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 69.7 MB/s  0:00:02m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 55.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 75.4 MB/s  0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.4.0
    Uninstalling triton-3.4.0:
      Successfully uninstalled triton-3.4.0━━━━━ 0/6 [triton]
  Attempting uninstall: nvidia-nccl-cu12━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/6 [nvidia-nvshmem-cu12]
    Found existing installation: nvidia-nccl-cu12 2.27.3━━━━━━ 1/6 [nvidia-nvshmem-cu12]
    Uninstalling nvidia-nccl-cu12-2.27.3:━━━━━━━━━━━━━━━━━━━━━ 1/6 [nvidia-nvshmem-cu12]
      Successfully uninstalled nvidia-nccl-cu12

In [ ]:
import os
import torch
from transformers import ( 
    AutoTokenizer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)
from datasets import Dataset, Audio
import pandas as pd
import torch
from torch import nn 
from transformers.modeling_outputs import BaseModelOutput
import torch.nn as nn


# Disable tokenizers parallelism to stop the warning
os.environ["TOKENIZERS_PARALLELISM"] = "false"

### Load Data

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.2G  100 10.2G    0     0  37.5M      0  0:04:39  0:04:39 --:--:-- 48.4M


In [7]:
!unzip -q workspace/CapstoneDataV2.zip -d /Data/

In [8]:
#rm -rf ~/.local/share/Trash/files/*
#rm -rf ~/.local/share/Trash/info/*


In [9]:
AUDIO_PATH = 'Data/content/Dataset/Audio'
SUMMARY_PATH = 'Data/content/Dataset/Summary'

In [10]:
data = []
for fname in os.listdir(AUDIO_PATH):
    if fname.endswith(".mp3"):
        base = os.path.splitext(fname)[0]
        summary_file = os.path.join(SUMMARY_PATH, f"SUM_{base}.txt")
        if os.path.exists(summary_file):
            with open(summary_file, 'r', encoding='utf-8') as f:
                summary = f.read().strip()
            data.append({
                "audio": os.path.join(AUDIO_PATH, fname),
                "summary": summary
            })

In [ ]:
df = pd.DataFrame(data)
dataset = Dataset.from_pandas(df)
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
dataset

Dataset({
    features: ['audio', 'summary'],
    num_rows: 9004
})

### Load Model

In [4]:
from transformers import AutoModelForSpeechSeq2Seq
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    "huypd1508/wav2vec2-t5-Vietnamese",
    trust_remote_code = True
)

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

(…)iguration_wav2vec2_to_vit5_bottleneck.py:   0%|          | 0.00/647 [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/huypd1508/wav2vec2-t5-Vietnamese:
- configuration_wav2vec2_to_vit5_bottleneck.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_wav2vec2_to_vit5_bottleneck.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/huypd1508/wav2vec2-t5-Vietnamese:
- modeling_wav2vec2_to_vit5_bottleneck.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/904M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/904M [00:00<?, ?B/s]

Some weights of the model checkpoint at huypd1508/wav2vec2-t5-Vietnamese were not used when initializing Wav2Vec2ToVit5WithAdapter: ['bottleneck.bottleneck_proj.bias', 'bottleneck.bottleneck_proj.weight', 'bottleneck.encoder.layers.0.linear1.bias', 'bottleneck.encoder.layers.0.linear1.weight', 'bottleneck.encoder.layers.0.linear2.bias', 'bottleneck.encoder.layers.0.linear2.weight', 'bottleneck.encoder.layers.0.norm1.bias', 'bottleneck.encoder.layers.0.norm1.weight', 'bottleneck.encoder.layers.0.norm2.bias', 'bottleneck.encoder.layers.0.norm2.weight', 'bottleneck.encoder.layers.0.self_attn.in_proj_bias', 'bottleneck.encoder.layers.0.self_attn.in_proj_weight', 'bottleneck.encoder.layers.0.self_attn.out_proj.bias', 'bottleneck.encoder.layers.0.self_attn.out_proj.weight', 'bottleneck.encoder.layers.1.linear1.bias', 'bottleneck.encoder.layers.1.linear1.weight', 'bottleneck.encoder.layers.1.linear2.bias', 'bottleneck.encoder.layers.1.linear2.weight', 'bottleneck.encoder.layers.1.norm1.bias',

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=32,             
    lora_alpha=64,      
    lora_dropout=0.1,
  
    target_modules=["q_proj", "v_proj", "q", "v"],
    modules_to_save=["adapter"], 
    bias="none"
)

#Apply LoRA
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 6,692,352 || all params: 328,988,800 || trainable%: 2.0342


In [6]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor.from_pretrained("nguyenvulebinh/wav2vec2-base-vietnamese-250h")
tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base-vietnews-summarization")

preprocessor_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [12]:
split = dataset.train_test_split(test_size=0.05,seed=42)
train_dataset = split['train']
eval_dataset = split['test']

In [ ]:
# Load one sample 

example = dataset[0]
print(example.keys())


print("Raw audio shape:", example["audio"]["array"].shape)
print("Summary text:", example["summary"])

dict_keys(['audio', 'summary'])
Raw audio shape: (3019968,)
Summary text: Làng giải trí Hàn Quốc xôn xao trước thông tin Kim Woo Bin chi 10 triệu USD để mua nhà. Việc nhóm nhạc nam nổi tiếng Hàn Quốc BTS mua lại trụ sở đã gây ra nhiều tranh cãi trong cộng đồng người hâm mộ. Sau thời gian chống chọi với bệnh tật, diễn viên Kim Woo Bin xuất hiện trở lại với màn ảnh nhỏ. Sau bao sóng gió, cuối cùng Kim Woo Bin và bạn gái kém 5 tuổi đã về một nhà và sống chung như một gia đình.


### Training

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./w2v_t5_lora",
    max_steps=18001,                     
    per_device_train_batch_size=1,         
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,   
    dataloader_num_workers=8, 
    dataloader_pin_memory=True,

    learning_rate=3e-4,                     
    warmup_ratio=0.1,
    weight_decay=0.01,

    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=100,                     

    eval_strategy="steps",
    eval_steps=3000,
    save_strategy="steps",
    save_steps=3000,                
    save_total_limit=2,                     
    load_best_model_at_end=True,
    metric_for_best_model="eval_rougeL", 
    greater_is_better=True,
    bf16=True,
    label_smoothing_factor=0.1,

    predict_with_generate=True,
    generation_max_length=256,
    generation_num_beams=4,                
    report_to=["tensorboard"],
    remove_unused_columns=False,
    save_safetensors=False
)

In [ ]:
def custom_data_collator(features):
    #  Raw audio and summaries
    raw_audio = [f["audio"]["array"] for f in features]
    summaries = [f["summary"] for f in features]

    #  Audio -> input features
    audio_features = processor(
        raw_audio,
        sampling_rate=16000,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=960000*5,   # 5 minutes @ 16kHz
        return_attention_mask=True,
    )

    #  Summaries -> tokenized
    tokenized = tokenizer(
        summaries,
        padding="max_length",
        max_length=256,
        truncation=True,
        return_tensors="pt",
    )

    labels = tokenized.input_ids
    labels[labels == tokenizer.pad_token_id] = -100

    return {
        "input_values": audio_features["input_values"],  
        "attention_mask": audio_features["attention_mask"],
        "labels": labels,
    }

In [ ]:
import evaluate
import numpy as np
from transformers import EarlyStoppingCallback

rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")


def compute_metrics(eval_preds):
    predictions, labels = eval_preds
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=False) 
    result = {key: value * 100 for key, value in rouge_result.items()} 
    
    
    return result




In [ ]:


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,         
    data_collator=custom_data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

/tmp/ipykernel_1399/3227549229.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
3000,5.635400,2.580914,35.658363,10.519820,22.913981,22.906789
6000,5.295800,2.489922,38.418914,11.430166,23.316525,23.313039
9000,5.114000,2.445480,41.995131,13.129374,24.365131,24.405420
12000,5.094300,2.406993,45.271840,14.779206,26.474547,26.483396


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

# Evaluate

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Pick one sample 
sample = dataset[2]  
batch = custom_data_collator([sample]) 

input_features = batch["input_features"].to(device)
attention_mask = batch["attention_mask"].to(device)

# Generate summary 
with torch.no_grad():
    outputs = model.generate(
        input_values=input_features,
        attention_mask=attention_mask,
        max_length=256,
    )

# Comparison 
pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
real = tokenizer.decode([t for t in batch["labels"][0].tolist() if t != -100], skip_special_tokens=True)

print("🟢 Generated Summary:", pred)
print("🔵 Real Summary:", real)

🟢 Generated Summary: Một người đàn ông Mỹ đã bị bắt sau khi nói dối rằng cô không phải là người yêu cũ của mình. Vụ bắt giữ cựu Tổng thống Mỹ Donald Trump được cho là nhằm vào một nhân vật có liên quan đến vụ ám sát hụt khiến ít nhất 10 người thiệt mạng.
🔵 Real Summary: Người phụ nữ đội mũ đen phía sau ông Trump được cho là có hành vi kỳ lạ khi vụ nổ súng vang lên. Vụ xả súng giết chết một khán giả vào ngày 21/7 có thể là động cơ khiến người phụ nữ trong phim Terminator 2 : Judgment Day ( Kẻ giết người ) trở thành hung thủ. Các đặc vụ Mỹ đã phá được các cửa hàng súng của công ty vũ khí Mỹ ở Mỹ sau khi phát hiện vụ nổ súng làm bị thương rất nhiều người. Bạn nghĩ rằng mình sống như thế nào?


### Uploading


In [30]:
from huggingface_hub import notebook_login
notebook_login()



In [22]:
trainer.push_to_hub("huypd1508/wav2vec2-t5-Vietnamese")

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /wav_vit_vietnamese/pytorch_model.bin :   0%|          |  555kB / 1.29GB            

  /wav_vit_vietnamese/training_args.bin :  15%|#5        |   908B / 5.97kB            

CommitInfo(commit_url='https://huggingface.co/huypd1508/wav_vit_vietnamese/commit/fefd2f22b1530b023604daabe3876128c4f979a0', commit_message='huypd1508/wav2vec2-t5-Vietnamese', commit_description='', oid='fefd2f22b1530b023604daabe3876128c4f979a0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/huypd1508/wav_vit_vietnamese', endpoint='https://huggingface.co', repo_type='model', repo_id='huypd1508/wav_vit_vietnamese'), pr_revision=None, pr_num=None)

In [33]:
from huggingface_hub import upload_folder,create_repo
create_repo('huypd1508/wav2vec2-t5-checkpoint-3000')

RepoUrl('https://huggingface.co/huypd1508/wav2vec2-t5-checkpoint-3000', endpoint='https://huggingface.co', repo_type='model', repo_id='huypd1508/wav2vec2-t5-checkpoint-3000')

In [34]:
from huggingface_hub import upload_folder,create_repo

upload_folder(
    repo_id='huypd1508/wav2vec2-t5-checkpoint-3000',
    folder_path="./wav_vit_vietnamese/checkpoint-3000",
    path_in_repo=".",  
    repo_type="model"
)

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...e/checkpoint-3000/training_args.bin: 100%|##########| 5.97kB / 5.97kB            

  ...namese/checkpoint-3000/scheduler.pt: 100%|##########| 1.47kB / 1.47kB            

  ...ietnamese/checkpoint-3000/scaler.pt: 100%|##########| 1.38kB / 1.38kB            

  ...amese/checkpoint-3000/rng_state.pth: 100%|##########| 14.7kB / 14.7kB            

  ...e/checkpoint-3000/pytorch_model.bin:   6%|5         | 75.4MB / 1.29GB            

  ...namese/checkpoint-3000/optimizer.pt:   7%|7         | 83.9MB / 1.14GB            

CommitInfo(commit_url='https://huggingface.co/huypd1508/wav2vec2-t5-checkpoint-3000/commit/bd7ab3d758bcf9f3e40baf1458fff73b19b7cb34', commit_message='Upload folder using huggingface_hub', commit_description='', oid='bd7ab3d758bcf9f3e40baf1458fff73b19b7cb34', pr_url=None, repo_url=RepoUrl('https://huggingface.co/huypd1508/wav2vec2-t5-checkpoint-3000', endpoint='https://huggingface.co', repo_type='model', repo_id='huypd1508/wav2vec2-t5-checkpoint-3000'), pr_revision=None, pr_num=None)